In [3]:
import sqlite3
import pandas as pd
import numpy as np

import datetime

<b> Old Billboard Data </b> <br>
We scraped this idea, keeping it here just in case 

In [3]:
# SCRAPE THE BILLBOARD 200 DATA

# connect with database
conn = sqlite3.connect("billboard-200.db")

# cursor is temp work area for manipulating SQL stmts
#cursor = conn.cursor()

# retrieve data from the 'album' table
album_query = "SELECT * FROM albums"
albums = pd.read_sql(album_query, conn) 

#album_query = cursor.execute()

# retireve data from the 'acoustic_features' table
features_query = "SELECT * FROM acoustic_features"
acoustic_features = pd.read_sql(features_query, conn) 

albums.head()

,id,date,artist,album,rank,length,track_length
0,1,None,None,None,None,NaN,NaN
1,2,2019-01-19,A Boogie Wit da Hoodie,Hoodie SZN,1,20.0,185233.800000
2,3,2019-01-19,21 Savage,I Am > I Was,2,15.0,211050.733333
3,4,2019-01-19,Soundtrack,Spider-Man: Into The Spider-Verse,3,13.0,190866.384615
4,5,2019-01-19,Meek Mill,Championships,4,19.0,219173.894737


In [ ]:
# we want to find the top 5 albums per year for the decades: 70s, 80s, 90s, 00s

# delete first row of albums dataframe 
top_albums = albums.iloc[1:]
top_albums.head()

In [4]:
acoustic_features.head()

,id,song,album,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,album_id,date
0,0Veyvc3n9AcLSoK3r1dA12,Voices In My Head,Hoodie SZN,A Boogie Wit da Hoodie,0.0555,0.754,142301.0,0.663,0.000000,6.0,0.101,-6.311,0.0,0.427,90.195,4.0,0.207,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
1,77JzXZonNumWsuXKy9vr3U,Beasty,Hoodie SZN,A Boogie Wit da Hoodie,0.2920,0.860,152829.0,0.418,0.000000,7.0,0.106,-9.061,0.0,0.158,126.023,4.0,0.374,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
2,18yllZD0TdF7ykcREib8Z1,I Did It,Hoodie SZN,A Boogie Wit da Hoodie,0.1530,0.718,215305.0,0.454,0.000046,8.0,0.116,-9.012,1.0,0.127,89.483,4.0,0.196,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
3,1wJRveJZLSb1rjhnUHQiv6,Swervin (feat. 6ix9ine),Hoodie SZN,A Boogie Wit da Hoodie,0.0153,0.581,189487.0,0.662,0.000000,9.0,0.111,-5.239,1.0,0.303,93.023,4.0,0.434,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
4,0jAfdqv18goRTUxm3ilRjb,Startender (feat. Offset and Tyga),Hoodie SZN,A Boogie Wit da Hoodie,0.0235,0.736,192779.0,0.622,0.000000,6.0,0.151,-4.653,0.0,0.133,191.971,4.0,0.506,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21


In [5]:
conn.close()

#### New Version of cleaning data base
bear with me 

thought process, merging the dataframe and deleting duplicates before deleting the exrta years (save this as a separate database to use when making the visualizations)

thennn delete the unnecessary years as preparation for sentinent analysis sampling :)

In [3]:
# read into dataframes
features_df = pd.read_csv('Hot 100 Audio Features.csv')
ranks_df = pd.read_csv('Hot Stuff.csv')

# make copies
features = features_df.copy(deep=True)
ranks = ranks_df.copy(deep=True)

In [4]:
# ranks preprocessing
dt = []
for row_index, row in ranks.iterrows():
    temp = datetime.datetime.strptime(ranks.at[row_index, 'WeekID'], '%m/%d/%Y')
    dt.append(temp)
ranks['Datetime'] = dt

ranks = ranks.sort_values(by=['Datetime', 'Week Position'])

In [5]:
# explain how we are going to do the merge 
# merge ranks_sample and features dataframe into songs data frame
songs = pd.merge(ranks, features, on='SongID', how='inner')

# clean up songs dataframe by removing columns we wont need 
cols_remove = ['Performer_y', 'Song_y', 'Weeks on Chart', 'Peak Position', 'Previous Week Position', 'Instance']
songs.drop(cols_remove, axis = 1, inplace=True)

songs.rename(columns={"Performer_x": "Performer", "Song_x": "Song"}, inplace=True)

# lets remove rows with duplicate SongIDs just for funsies 
songs = songs.drop_duplicates(subset='SongID', keep="first")
songs.reset_index(drop=True, inplace=True)

# add a separate row with only the year
# doing this will help us create visualizations later
years = []
for row_index, row in songs.iterrows(): 
    years.append(row["Datetime"].year)
    
songs['Year'] = years

Now that we have cleaned up our databases by adding useful columns, deleting unnecessary ones, and merging our rank and features, lets start preparing the songs database for sampling. 

give an explanation of how we are going to sample 

In [7]:
songs_sample= songs.copy(deep=True)
songs_sample.reset_index(drop=True, inplace=True)

In [8]:
# remove decades that arent complete (50s, 2010s)

years_1 = np.arange(1958, 1960)
years_2 = np.arange(2011, 2020)
years_removed = np.concatenate((years_1, years_2), axis=None)

In [9]:
bad_rows = []

for row_index, row in songs_sample.iterrows(): 
    if row['Datetime'].year in years_removed:
        bad_rows.append(row_index)

In [10]:
songs_sample.drop(songs_sample.index[bad_rows], inplace=True)
songs_sample.reset_index(drop=True, inplace=True)

In [11]:
songs_sample

,url,WeekID,Week Position,Song,Performer,SongID,Datetime,spotify_genre,spotify_track_id,spotify_track_preview_url,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Year
0,http://www.billboard.com/charts/hot-100/1960-0...,1/2/1960,68,Crazy Arms,Bob Beckham,Crazy ArmsBob Beckham,1960-01-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1960
1,http://www.billboard.com/charts/hot-100/1960-0...,1/2/1960,78,Skokiaan (South African Song),Bill Haley And His Comets,Skokiaan (South African Song)Bill Haley And Hi...,1960-01-02,"['adult standards', 'brill building pop', 'doo...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1960
2,http://www.billboard.com/charts/hot-100/1960-0...,1/2/1960,82,Rockin' Little Angel,Ray Smith,Rockin' Little AngelRay Smith,1960-01-02,"['rock-and-roll', 'rockabilly']",1R4kAzLGI0PQBe32NAGP40,https://p.scdn.co/mp3-preview/ab8c5d25a04cb0dd...,...,-5.065,1.0,0.0796,0.8040,0.000002,0.1350,0.9620,79.156,4.0,1960
3,http://www.billboard.com/charts/hot-100/1960-0...,1/2/1960,84,Tracy's Theme,Spencer Ross,Tracy's ThemeSpencer Ross,1960-01-02,[],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1960
4,http://www.billboard.com/charts/hot-100/1960-0...,1/2/1960,87,Riverboat,Faron Young,RiverboatFaron Young,1960-01-02,"['classic country pop', 'honky tonk', 'rockabi...",1LXTz7SxX7C7SCoFpz0Tc2,NaN,...,-8.817,1.0,0.0490,0.4910,0.000000,0.0990,0.9420,90.851,4.0,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23511,http://www.billboard.com/charts/hot-100/2010-1...,12/25/2010,59,Welcome Christmas,Glee Cast,Welcome ChristmasGlee Cast,2010-12-25,"['glee club', 'hollywood', 'post-teen pop']",453OW4naq3cgZlk9oaHCu9,https://p.scdn.co/mp3-preview/802b81ffd968fcf2...,...,-9.302,1.0,0.0297,0.9110,0.000000,0.2750,0.0819,110.822,4.0,2010
23512,http://www.billboard.com/charts/hot-100/2010-1...,12/25/2010,68,Rolling In The Deep,Adele,Rolling In The DeepAdele,2010-12-25,"['british soul', 'pop', 'uk pop']",1CkvWZme3pRgbzaxZnTl5X,https://p.scdn.co/mp3-preview/8c174ce25d94b502...,...,-5.119,1.0,0.0294,0.1310,0.000000,0.0527,0.5220,104.945,4.0,2010
23513,http://www.billboard.com/charts/hot-100/2010-1...,12/25/2010,89,No BS,Chris Brown,No BSChris Brown,2010-12-25,"['dance pop', 'pop', 'pop rap', 'r&b', 'rap']",6P3cUSAoT4EeMxVgWVdBra,https://p.scdn.co/mp3-preview/4b3158b2f9d396b8...,...,-6.359,0.0,0.0567,0.1200,0.000000,0.1190,0.1060,172.171,4.0,2010
23514,http://www.billboard.com/charts/hot-100/2010-1...,12/25/2010,94,Bass Down Low,Dev Featuring The Cataracs,Bass Down LowDev Featuring The Cataracs,2010-12-25,"['dance pop', 'electropop']",4Xf7EL8G6FRYwvJADl0nf4,NaN,...,-8.022,1.0,0.0628,0.0886,0.000000,0.0499,0.8340,113.035,4.0,2010


## FULL CODE FOR WORD CLOUD 

In [2]:
# wordcloud libraries 
!pip install wordcloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import warnings
warnings.filterwarnings("ignore")

In [8]:
songs_sampled = pd.read_csv("Songs_Sampled.csv")


In [ ]:
cloud_df = songs_sampled.copy(deep=True)
cloud_df = cloud_df.sort_values(by='Year')
cloud_df.reset_index(inplace=True)

In [ ]:
lyrics_60s = ""
for row_index, row in songs_sampled[0:100].iterrows():
    song = songs_sampled.at[row_index, 'Song']
    artist = songs_sampled.at[row_index, 'Performer']
    temp = get_lyrics(artist, song)
    if (temp is not None):
        lyrics_60s += temp 

In [ ]:
lyrics_70s = ""
for row_index, row in songs_sampled[100:200].iterrows():
    song = songs_sampled.at[row_index, 'Song']
    artist = songs_sampled.at[row_index, 'Performer']
    temp = get_lyrics(artist, song)
    if (temp is not None):
        lyrics_70s += temp 

In [ ]:
lyrics_80s = ""
for row_index, row in songs_sampled[200:300].iterrows():
    song = songs_sampled.at[row_index, 'Song']
    artist = songs_sampled.at[row_index, 'Performer']
    temp = get_lyrics(artist, song)
    if (temp is not None):
        lyrics_80s += temp 

In [ ]:
lyrics_90s = ""
for row_index, row in songs_sampled[300:400].iterrows():
    song = songs_sampled.at[row_index, 'Song']
    artist = songs_sampled.at[row_index, 'Performer']
    temp = get_lyrics(artist, song)
    if (temp is not None):
        lyrics_90s += temp 

In [ ]:
lyrics_00s = ""
for row_index, row in songs_sampled[400:500].iterrows():
    song = songs_sampled.at[row_index, 'Song']
    artist = songs_sampled.at[row_index, 'Performer']
    temp = get_lyrics(artist, song)
    if (temp is not None):
        lyrics_00s += temp 

In [ ]:
# Generate word cloud for songs from 1958-1960
text = lyrics_60s

# Create and generate a word cloud image:
wordcloud_60s = WordCloud(max_words=100, background_color="white").generate(text)

# Display the generated image:
plt.title("1960s")
plt.imshow(wordcloud_60s, interpolation='bilinear')
plt.axis("off")
plt.show()


# Generate word cloud for songs from the 1970s
text = lyrics_70s

# Create and generate a word cloud image:
wordcloud_70s = WordCloud(max_words=100, background_color="white").generate(text)

# Display the generated image:
plt.title("1970s")
plt.imshow(wordcloud_70s, interpolation='bilinear')
plt.axis("off")
plt.show()


# Generate word cloud for songs from the 1980s
text = lyrics_80s

# Create and generate a word cloud image:
wordcloud_80s = WordCloud(max_words=100, background_color="white").generate(text)

# Display the generated image:
plt.title("1980s")
plt.imshow(wordcloud_80s, interpolation='bilinear')
plt.axis("off")
plt.show()

# Generate word cloud for songs from the 1990s
text = lyrics_90s

# Create and generate a word cloud image:
wordcloud_90s = WordCloud(max_words=100, background_color="white").generate(text)

# Display the generated image:
plt.title("1990s")
plt.imshow(wordcloud_90s, interpolation='bilinear')
plt.axis("off")
plt.show()

# Generate word cloud for songs from the 2000s
text = lyrics_00s

# Create and generate a word cloud image:
wordcloud_00s = WordCloud(max_words=100, background_color="white").generate(text)

# Display the generated image:
plt.title("2000s")
plt.imshow(wordcloud_00s, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Save the image 
wordcloud_60s.to_file("wc_60s.png")

In [ ]:
# machine learning & regression
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [ ]:
# # plot x=acoustic feature, y=compound score

# x =feature_averages['speechiness']
# y =polarity_averages['Comp_Score']


# plt.scatter(x, y, alpha=0.5)
# plt.title('')
# plt.xlabel('speechiness')
# plt.ylabel('compound score')
# plt.show()

In [ ]:
# fit linear regression model compound score vs. year
# tells us the same thing & gives same equation as Mean Compound Score Per Year Plot 

# set up arrays for year and life expectancy
a = np.array(polarity_averages['Year'])
b = np.array(polarity_averages['Comp_Score'])

# transform columns into nd arrays
X = a.reshape((len(a), 1))
y = b.reshape((len(b), 1))

# fit linear model
reg = LinearRegression().fit(X, y)

# Coefficients
print('Coefficients: ', reg.coef_)

# Intercept
print('Intercept: ', reg.intercept_)

In [ ]:
# fit linear regression model compound score vs. danceability (example feature)

# set up arrays for year and life expectancy
a = np.array(feature_averages['danceability'])
b = np.array(polarity_averages['Comp_Score'])

# transform columns into nd arrays
X = a.reshape((len(a), 1))
y = b.reshape((len(b), 1))

# fit linear model
reg = LinearRegression().fit(X, y)

# Coefficients
print('Coefficients: ', reg.coef_)

# Intercept
print('Intercept: ', reg.intercept_)

In [ ]:
# performing multiple linear regression

acoustic_features = ['danceability','energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                     'liveness', 'valence', 'tempo']

X = feature_averages[acoustic_features] 
y = polarity_averages['Comp_Score']

# fit regression model
regr = LinearRegression()
regr.fit(X, y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

# prediction with sklearn
new_danceability = 0.533456
new_energy = 0.492543
new_loudness = -10.674654
new_speechiness = 0.051123
new_acousticness = 0.765437
new_instrumentalness = 0.056744
new_liveness = 0.124577
new_valence = 0.665433
new_tempo = 114.788876

prediction_vals = [new_danceability, new_energy, new_loudness, new_speechiness, new_acousticness, 
                   new_instrumentalness, new_liveness, new_valence, new_tempo]

print ('Predicted Compound Score: \n', regr.predict([prediction_vals]))

# prediction with statsmodels (gives us lots of interesting info)
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

We want to predict the compound score (dependent variables) based on 9 acoustic features (independed variables): danceability, energy, loudness, speechiness, acousticness, intrumentalness, liveness, valence, and tempo 

buutt we end up with high multicolinearity cuz the independent variables are too correlated, so we would have to create interaction terms. Doing that for 9-10 variables would be insane, we would end up with too many interaction terms than would be feasible so lets pick a few acoustic features whose plots over time we found interesting.